In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
### PANEL GŁÓWNY ### - większość zmiennych wykonywanych w dalszej części kodu bierze się stąd. Szczegóły - zeszyt 1

observations = 1000  # Ustaw ilość obserwacji (Startowa: 1000)
noic = 10            # Szum w bazie przypadków treningowych.
noit = 1             # Szum wygenerowany dla funkcji testującej (docelowej).
model = [13,7,-12]   # Parametry modelu losowego dla funkcji a*xs + b*xz + c + szum. Definiujemy tu dwie docelowe wagi i obciążenie do których model ma dążyć.
                     # Można też rzecz jasna dłubać przy tym ręcznie, albo wyciągnąć tutaj całą funkcję i potem ją załadować ze zmiennej...

In [4]:
xs = np.random.uniform(low = -noic, high = noic, size =(observations,vnum)) # Gen. zakr. los. zm. wej.(patrz zeszyt 1)
zs = np.random.uniform(-noic,noic,(observations,vnum)) # J.w.- bez opisów bo zbędne. Poz. dla orientacji.

#inputs
generated_inputs = np.column_stack((xs,zs)) # Składanie z pow. matrycy w formacie observationsx2. (Patrz zeszyt 1)

noise = np.random.uniform(-noit,noit,(observations,1)) # Gen. zakr. los. (Patrz zeszyt 1)

generated_targets = model[0]*xs + model[1]*zs + model[2] + noise # Funkcja symulująca zadane cele. Do niej dąży model. (Patrz zeszyt 1)

#TF_Intro
np.savez('My_Nums', inputs = generated_inputs, targets = generated_targets)